In [104]:
import numpy;
import math;
import torch;

def getData(N, variance):
    """
    Generates a dataset {(xi, yi) : i = 1, 2, . . .N} of N (X, Y ) pairs for a given value of N and sigma^2
    :return: The generated dataset of (X,Y) pairs
    """
    X = numpy.random.rand(N) # N values between 0 and 1
    Z = numpy.random.normal(0, math.sqrt(variance), N) # Zero mean Gaussian random variable, with standard deviation, of length N
    Y = numpy.cos(2*math.pi*X) + Z # Y generated from function (1)
    print("X: ", X)
    print("Z: ", Z)
    print ("Y: ", Y)

    # store the (X,Y) pairs in a N x 2 matrix
    x_y_vector = numpy.column_stack((X, Y))
    print("(X,Y) pairs:", x_y_vector)
    return x_y_vector
    
def getMSE(y_predicted, y_true): # this is probably wrong
    """
    Get the Mean Square Error (MSE) loss from the predicted dataset, and the true dataset
    :return: The Mean Square Error (MSE)
    """
    # MSE calculation using numpy, but will manually calculate below
    mse = numpy.mean((y_true - y_predicted)**2) 
    print ("Automatic MSE:", mse)

    # Sum all the squared differences of each y_pred, y_true. Then average that over the vector length, N 
    N = len(y_predicted)
    summation_diffs_squared = 0
    for y_pred_i, y_true_i in zip(y_predicted, y_true):
        squared_diff = (y_pred_i - y_true_i) ** 2
        summation_diffs_squared += squared_diff
    mse = summation_diffs_squared / N
    print ("Calculated MSE: ", mse)
    return mse

def fitData(data_set, degree_poly):
    """
    fitData method that estimates the polynomial coefficients by fitting a given data to a degree-d polynomial
    :return: estimated_coefficients, Ein (MSE), Eout
    """

    # Random starting coefficents
    starting_coeff = torch.tensor(startingRandomCoeffs(degree_poly))
    print("Random starting coeff:", starting_coeff)



    Ein = 2.345
    Eout = 4.567
    return starting_coeff, Ein, Eout # this should be changed to the eventual estimated coefficients

def startingRandomCoeffs(d):
    """
    Get starting random coefficients, a0, a1, ..., ad, for the polynomial regression function
    :return: Randome coefficients for the polynomial based on random values between 0 and 1
    """
    return numpy.random.rand(d)





 



In [105]:

N = 10          # Size of the training dataset
variance = 0.2  # Variance, sigma^2
degree_d=3      # Degree of the model polynomial


data_set = getData(N, variance)
MSE = getMSE(numpy.array([2,34,4]), numpy.array([1.9,37,4.3]))

fitData(data_set, degree_d)



X:  [0.43075067 0.19334026 0.3203018  0.31244481 0.73846116 0.06456755
 0.47468698 0.9753679  0.33797114 0.86466854]
Z:  [-0.23648355  0.56390411 -0.75069394 -0.0011535   0.13661878  0.1874676
  0.95243844  0.98960109 -0.14310519 -0.39592958]
Y:  [-1.14330877  0.91243539 -1.17818828 -0.38351652  0.06418164  1.10629794
 -0.03494032  1.97764841 -0.66812552  0.26381865]
(X,Y) pairs: [[ 0.43075067 -1.14330877]
 [ 0.19334026  0.91243539]
 [ 0.3203018  -1.17818828]
 [ 0.31244481 -0.38351652]
 [ 0.73846116  0.06418164]
 [ 0.06456755  1.10629794]
 [ 0.47468698 -0.03494032]
 [ 0.9753679   1.97764841]
 [ 0.33797114 -0.66812552]
 [ 0.86466854  0.26381865]]
Automatic MSE: 3.033333333333333
Calculated MSE:  3.033333333333333
Random coeff: tensor([0.2384, 0.3363, 0.6733], dtype=torch.float64)


(tensor([0.2384, 0.3363, 0.6733], dtype=torch.float64), 2.345, 4.567)

In [106]:
M = 35      # Number of trials

# -- playing with pytorch
x = torch.tensor(3.0, requires_grad = True)
print("creating a tensor x: ", x)

y = 3*x**2
print("Equation result", y)
y.backward()
print("Derivative:", x.grad)





creating a tensor x:  tensor(3., requires_grad=True)
Equation result tensor(27., grad_fn=<MulBackward0>)
Derivative: tensor(18.)




d



d

You can calculate the Mean Squared Error (MSE) for a polynomial regression without using external libraries by implementing the calculations manually. Here's how you can do it step by step:

# Sample dataset (replace with your own data)
X = np.array([1, 2, 3, 4, 5])
y = np.array([2.1, 4.0, 7.2, 9.1, 10.8])

# Specify the polynomial degree
degree = 2  # You can change this to the desired degree

# Create polynomial features
X_poly = np.column_stack([X**i for i in range(degree + 1)])

# Calculate the coefficients of the polynomial using linear regression
coefficients = np.linalg.inv(X_poly.T @ X_poly) @ X_poly.T @ y

# Make predictions
y_pred = X_poly @ coefficients

# Calculate the Mean Squared Error (MSE) manually
mse = np.mean((y - y_pred) ** 2)

print(f"Mean Squared Error (MSE): {mse}")


In this code:

We create the polynomial features by raising the input values to powers from 0 to the specified degree (degree).
We calculate the coefficients of the polynomial using the normal equation of linear regression without an external library.
We make predictions using the calculated coefficients.
Finally, we manually calculate the MSE by taking the mean of the squared differences between the actual target values (y) and the predicted values (y_pred).
This code performs the polynomial regression and computes the MSE without relying on external libraries for these calculations.